In [3]:
import pandas as pd
import numpy as np

# caminho CORRETO a partir da pasta notebooks
df = pd.read_csv("../data/full_warehouse_merged.csv")

# valores que representam nulo
null_values = ["-1", -1, "no_data", "sem_registro"]
df = df.replace(null_values, np.nan)

# 1) alcohol_freq → categoria "unknown"
df["alcohol_freq"] = df["alcohol_freq"].fillna("unknown")

# 2) bins de idade
age_bins = pd.cut(df["age"], bins=[0, 18, 30, 45, 65, 120])

# 3) education por faixa etária
df["education"] = df.groupby(age_bins)["education"].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "unknown")
)

# 4) marital_status por faixa etária
df["marital_status"] = df.groupby(age_bins)["marital_status"].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "unknown")
)

# 5) sex pela moda geral
df["sex"] = df["sex"].fillna(df["sex"].mode()[0])

# 6) network_tier pela moda geral
df["network_tier"] = df["network_tier"].fillna(df["network_tier"].mode()[0])

# 7) biométricas por mediana condicional (sexo + faixa etária)
biom = ["bmi", "systolic_bp", "diastolic_bp", "ldl", "hba1c"]

for col in biom:
    df[col] = df.groupby(["sex", age_bins])[col].transform(
        lambda x: x.fillna(x.median())
    )

# 8) income por mediana condicional (education + marital_status)
df["income"] = df.groupby(["education", "marital_status"])["income"].transform(
    lambda x: x.fillna(x.median())
)

# salvar dataset tratado na pasta data
output_path = "../data/full_warehouse_method_B_sem_sklearn.csv"
df.to_csv(output_path, index=False)

print("Imputação avançada concluída!")
print("Arquivo salvo em:", output_path)


/var/folders/99/0hmnt5l13rn3b4mhfw326ws00000gn/T/ipykernel_10902/158959086.py:18: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["education"] = df.groupby(age_bins)["education"].transform(
/var/folders/99/0hmnt5l13rn3b4mhfw326ws00000gn/T/ipykernel_10902/158959086.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["marital_status"] = df.groupby(age_bins)["marital_status"].transform(
/var/folders/99/0hmnt5l13rn3b4mhfw326ws00000gn/T/ipykernel_10902/158959086.py:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to r

Imputação avançada concluída!
Arquivo salvo em: ../data/full_warehouse_method_B_sem_sklearn.csv
